# Correlation Recommendation System

### This is an example for measuring the correlation of items based on their ratings

In [1]:
import numpy as np
import pandas as pd
import os
base_dir = "datasets\\"

In [8]:
frame = pd.read_csv(os.path.join(base_dir,"rating_final.csv"))
cuisine = pd.read_csv(os.path.join(base_dir,"chefmozcuisine.csv"))
geodata = pd.read_csv(os.path.join(base_dir,"geoplaces2.csv"),encoding = "ISO-8859-1")

In [9]:
frame.head()

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2


In [11]:
geodata.head()
places = geodata[['placeID','name']]
places.head()

,placeID,name
0,134999,Kiku Cuernavaca
1,132825,puesto de tacos
2,135106,El Rincón de San Francisco
3,132667,little pizza Emilio Portes Gil
4,132613,carnitas_mata


In [12]:
cuisine.head()

,placeID,Rcuisine
0,135110,Spanish
1,135109,Italian
2,135107,Latin_American
3,135106,Mexican
4,135105,Fast_Food


## Grouping and Ranking Data

In [17]:
rating = pd.DataFrame(frame.groupby('placeID')['rating'].mean())
rating.head()

,rating
placeID,
132560,0.50
132561,0.75
132564,1.25
132572,1.00
132583,1.00


In [18]:
rating['rating_count'] = pd.DataFrame(frame.groupby('placeID')['rating'].count())
rating.head()

,rating,rating_count
placeID,,
132560,0.50,4
132561,0.75,4
132564,1.25,4
132572,1.00,15
132583,1.00,4


In [20]:
# Get some statistics
rating.describe()

,rating,rating_count
count,130.000000,130.000000
mean,1.179622,8.930769
std,0.349354,6.124279
min,0.250000,3.000000
25%,1.000000,5.000000
50%,1.181818,7.000000
75%,1.400000,11.000000
max,2.000000,36.000000


In [21]:
# Find the place with most ratings
rating.sort_values('rating_count',ascending=False).head()

,rating,rating_count
placeID,,
135085,1.333333,36
132825,1.281250,32
135032,1.178571,28
135052,1.280000,25
132834,1.000000,25


In [22]:
# Find the name of the place
places[places['placeID']==135085]


,placeID,name
121,135085,Tortas Locas Hipocampo


In [24]:
# Find the cuisine of the place
cuisine[cuisine['placeID']==135085]

,placeID,Rcuisine
44,135085,Fast_Food


## Preparing Data for Analysis